In [254]:
import os
import re
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer

In [255]:
syllable_dict = pd.read_table("data/Syllable_dictionary.txt", header = None)

In [258]:
syllable_dictdict

,0
0,'gainst 1
1,'greeing E1 2
2,'scaped 1
3,'tis 1
4,'twixt 1
...,...
3200,yours 1
3201,youth 1
3202,youth's 1
3203,youthful 2


In [256]:
sylls = pd.DataFrame(0, index=np.arange(len(syllable_dict)), columns=["word", "0", "1", "E1", "2", "E2", "3", "E3", "4", \
                                                                      "E4", "5", "E5"])


In [257]:
for i in range(len(syllable_dict)):
    sp_string = syllable_dict[0][i].split()
    if sp_string[0][0] == '\'':
        sp_string[0] = sp_string[0][1: len(sp_string[0])]
    sylls["word"][i] = sp_string[0] 
    for j in range(1, len(sp_string)):
        sylls[sp_string[j]][i] = 1

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [81]:
shakespeare = pd.read_table("data/shakespeare.txt", names = ["sentences"])

In [222]:
indices = list(range(0, len(shakespeare), 15))

In [223]:
# Delete these row indexes from dataFrame
shakespeare.drop(indices , inplace=True)

In [225]:
def parse_observations(text):
    # Convert text to dataset.
    lines = [line.split() for line in text.split('\n') if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []
        
        for word in line:
            word = re.sub(r'[^\w]', '', word).lower()
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map

def obs_map_reverser(obs_map):
    obs_map_r = {}

    for key in obs_map:
        obs_map_r[obs_map[key]] = key

    return obs_map_r


In [133]:
def ie_preprocess(sentence):
    if (any(char.isdigit() for char in sentence)):
        return
    tknzr = TweetTokenizer()
    sentences = tknzr.tokenize(sentence)
    sentences = [w.lower() for w in sentences if not w in string.punctuation]
    return sentences

In [134]:
preprocessed = []
for item in list(shakespeare["sentences"]):
    returned = ie_preprocess(item)
    if returned != None:
        preprocessed.append(returned)

In [204]:
syl_map = {}
for line in preprocessed:
    for j in range(len(line)):
        one_hot = sylls[sylls['word'].str.find(line[j]) == 0].iloc[0]
        ser = one_hot.T.squeeze()
        ind = ser[ser == 1].index[0]
        if 'E' in ind:
            if j == len(line) - 1:
                ind = ind[1]
            else:
                ind = ser[ser == 1].index[1]
        syl_map[line[j]] = ind

In [233]:
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()


In [234]:
obs, obs_map = parse_observations(text)


In [237]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
# load text
raw_text = load_doc('data/shakespeare.txt')
# clean
raw_text = ''.join([i for i in raw_text if not i.isdigit()])
tokens = raw_text.split()
raw_text = ' '.join(tokens).lower()

In [249]:
from collections import Counter
result = dict(Counter(''.join([c.lower() for c in raw_text if c.isupper()])))

In [251]:
factor = 1.0/sum(result.values())
for k in result:
    result[k] = result[k]*factor

In [253]:
dump(result, open('first_freqs.pkl', 'wb'))